In [15]:
import numpy as np
import pandas as pd
from sklearn import datasets
from random import seed
from random import randrange
# importing StandardScaler library        
from sklearn.preprocessing import StandardScaler

In [16]:
class PolynomialRegression:
    def __init__(self, degree=2, lr=0.001, n_iters=1000):
        self.degree = degree
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.scaler = None

    def fit(self, X, y, sample_weight=None):
        # Add polynomial features
        X_poly = self._polynomial_features(X, self.degree)

        # Normalize data
        self.scaler = StandardScaler()
        X_poly = self.scaler.fit_transform(X_poly)

        n_samples, n_features = X_poly.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X_poly, self.weights) + self.bias

            dw = (1/n_samples) * np.dot(X_poly.T, (y_pred - y))
            db = (1/n_samples) * np.sum(y_pred - y)

            self.weights = self.weights - self.lr * dw
            self.bias = self.bias - self.lr * db

    def predict(self, X):
        # Add polynomial features
        X_poly = self._polynomial_features(X, self.degree)

        # Normalize data
        X_poly = self.scaler.transform(X_poly)

        y_pred = np.dot(X_poly, self.weights) + self.bias
        return y_pred

    def _polynomial_features(self, X, degree):
        n_samples, n_features = X.shape
        X_poly = np.ones((n_samples, 1))

        for d in range(1, degree+1):
            for i in range(n_features):
                X_poly = np.concatenate((X_poly, (X[:,i]**d).reshape(-1,1)), axis=1)

        return X_poly

In [17]:
#d = datasets.load_boston()
#x, y = d.data, d.target

# Load Boston dataset
boston = datasets.load_boston()
X, y = boston.data, boston.target

In [24]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

In [26]:
# Fit model and make predictions
reg = PolynomialRegression(degree=2, lr=0.01, n_iters=1000)
reg.fit(X_train, y_train)
predictions = reg.predict(X_test)

In [27]:
from sklearn.metrics import r2_score 
r2score = r2_score(y_test, predictions)
print('R2 score:', r2score)

R2 score: 0.7906880716392194


In [28]:
from itertools import product

# define hyperparameters to tune
lr = [0.001, 0.01, 0.1]
n_iters = [1000, 5000, 10000]

# create all possible combinations of hyperparameters
hyperparameters = list(product(lr, n_iters))

# initialize best accuracy and corresponding hyperparameters
best_r2score = 0
best_hyperparameters = None

# loop over all hyperparameters
for hyperparameter in hyperparameters:
    # create a new instance of LinearRegression with the current hyperparameters
    lr = PolynomialRegression(lr=hyperparameter[0], n_iters=hyperparameter[1])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)
    
    # fit the model
    lr.fit(X_train, y_train)
 
    
    # evaluate the model on the validation set
    y_pred = lr.predict(X_test)
    r2score=r2_score(y_test, predictions)
    
    # update best accuracy and corresponding hyperparameters
    if r2score > best_r2score:
        best_r2score= r2score
        best_hyperparameters = hyperparameter
        
# print the best hyperparameters and corresponding accuracy
print("Best hyperparameters:", best_hyperparameters)
print("Validation r2score:", best_r2score)

Best hyperparameters: (0.001, 1000)
Validation r2score: 0.7906880716392194
